# Meeting Compass

29/01/2020

Output files Compass for ROM with Kratos

## Example of MOR in Kratos: StanfordBunny

Two stages: 

1. Train MOR: Reduction degrees of freedom.

2. Train HROM: Reduction number of integration points. 

### Import libraries

In [1]:
import KratosMultiphysics
import KratosMultiphysics.RomApplication as romapp

from KratosMultiphysics.RomApplication.randomized_singular_value_decomposition import RandomizedSingularValueDecomposition
from KratosMultiphysics.StructuralMechanicsApplication.structural_mechanics_analysis import StructuralMechanicsAnalysis
from KratosMultiphysics.RomApplication.structural_mechanics_analysis_rom import StructuralMechanicsAnalysisROM
from matplotlib import pyplot as plt

import numpy as np
import json

### Define class to save data and create the Snapshot Matrix

In [2]:
class StructuralMechanicsAnalysisSavingData(StructuralMechanicsAnalysis):

    def __init__(self,model,project_parameters):
        super().__init__(model,project_parameters)
        self.time_step_solution_container = []

    def FinalizeSolutionStep(self):
        super().FinalizeSolutionStep()
        ArrayOfDisplacements = []
        for node in self._GetSolver().GetComputingModelPart().Nodes:
            ArrayOfDisplacements.append(node.GetSolutionStepValue(KratosMultiphysics.DISPLACEMENT_X, 0))
            ArrayOfDisplacements.append(node.GetSolutionStepValue(KratosMultiphysics.DISPLACEMENT_Y, 0))
            ArrayOfDisplacements.append(node.GetSolutionStepValue(KratosMultiphysics.DISPLACEMENT_Z, 0))
        self.time_step_solution_container.append(ArrayOfDisplacements)


    def EvaluateQuantityOfInterest(self):
       ##############################################################################################
       #     Functions evaluating the QoI of the problem: Snapshot matrix of every time step        #
       #                                                                                            #
       ##############################################################################################
        SnapshotMatrix = np.zeros((len(self.time_step_solution_container[0]), len(self.time_step_solution_container)))
        for i in range(len(self.time_step_solution_container)):
            Snapshot_i= np.array(self.time_step_solution_container[i])
            SnapshotMatrix[:,i] = Snapshot_i.transpose()
        return SnapshotMatrix

### Define parameters

In [3]:
with open("ProjectParameters.json",'r') as parameter_file:
    parameters = KratosMultiphysics.Parameters(parameter_file.read())

### Define model and simulation

In [4]:
model = KratosMultiphysics.Model()
simulation = StructuralMechanicsAnalysisSavingData(model,parameters)

### Run simulation

In [5]:
simulation.Run()

Solution saved in memory simulation.time_step_solution_container. It is not written in file. 

### Create snaposhot matrix

In [11]:
SnapshotMatrix = simulation.EvaluateQuantityOfInterest()

In [14]:
# Shape of this matrix
print('Shape of SnapshotMatrix = ', SnapshotMatrix.shape)

Shape of SnapshotMatrix =  (26688, 15)


In [16]:
# Number of displacements = 3 (displ_x, displ_y, displ_z)

# Number of nodes
print('Number of nodes =', SnapshotMatrix.shape[0] // 3)

# Number of time steps
print('Number of time steps = ', SnapshotMatrix.shape[1])

Number of nodes = 8896
Number of time steps =  15
